In [1]:
import json
import requests
import pandas as pd
import yfinance as yf
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Load session and CSRF token
with open("/Users/rakeshbali/Imperial/Stock Research/Shortlisted/StaticData/passwords.json", "r") as f:
    password = json.load(f)

session_id = password.get("session_id")
csrf_token = password.get("csrf_token")

# Headers
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Cookie': f'sessionid={session_id}',
    'X-CSRFToken': csrf_token
}

# Read index and URL data
df_links = pd.read_csv("nifty_index_urls.csv")

# Initialize empty dataframe
master_df = pd.DataFrame()

# Loop through each index
for _, row in df_links.iterrows():
    index_name = row['INDEX']
    url = row['URL']

    try:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            values = data.get('datasets')[0].get('values')
            print
            # Create dataframe with Date and Index Value
            temp_df = pd.DataFrame(values, columns=["Date", index_name])
            temp_df["Date"] = pd.to_datetime(temp_df["Date"])

            if master_df.empty:
                master_df = temp_df
            else:
                master_df = pd.merge(master_df, temp_df, on="Date", how="outer")
        else:
            print(f"Failed for {index_name}: Status Code {response.status_code}")
    except Exception as e:
        print(f"Error processing {index_name}: {e}")

# Sort by date
master_df = master_df.sort_values("Date")

# Save final combined data
master_df.to_csv("all_index_prices.csv", index=False)

Failed for NIFTY REALTY: Status Code 429
Failed for NIFTY CONSUMER DURABLES: Status Code 429
Failed for NIFTY OIL & GAS: Status Code 429
Failed for NIFTY100 LOW VOLATILITY 30: Status Code 429
Failed for NIFTY200 MOMENTUM 30: Status Code 429
Failed for NIFTY500 MOMENTUM 50: Status Code 429
Failed for NIFTY INDIA CONSUMPTION: Status Code 429
Failed for NIFTY CPSE: Status Code 429
Failed for NIFTY ENERGY: Status Code 429
Failed for NIFTY INFRASTRUCTURE: Status Code 429
Failed for NIFTY MNC: Status Code 429
Failed for NIFTY SERVICES SECTOR: Status Code 429
Failed for NIFTY INDIA DIGITAL: Status Code 429
Failed for NIFTY INDIA MANUFACTURING: Status Code 429
Failed for NIFTY INDIA DEFENCE: Status Code 429
Failed for NIFTY INDIA TOURISM: Status Code 429
Failed for NIFTY EV & NEW AGE AUTOMOTIVE: Status Code 429
Failed for NIFTY INDIA NEW AGE CONSUMPTION: Status Code 429
Failed for NIFTY MOBILITY: Status Code 429
Failed for NIFTY HOUSING: Status Code 429
Failed for NIFTY NON-CYCLICAL CONSUMER: 

In [3]:
df = pd.read_csv('all_index_prices.csv')

In [4]:
def calculate_technical_indicators_index(symbol):
    # Create a new DataFrame with just the date and symbol columns
    new_df = pd.DataFrame()
    new_df['Date'] = pd.to_datetime(df['Date'])
    new_df['Open'] = df[symbol].shift(1)
    new_df['Close'] = df[symbol]

    # Calculate moving averages
    new_df['5DMA'] = df[symbol].rolling(window=5, min_periods=1).mean()
    new_df['20DMA'] = df[symbol].rolling(window=20, min_periods=1).mean()
    new_df['50DMA'] = df[symbol].rolling(window=50, min_periods=1).mean()
    new_df['100DMA'] = df[symbol].rolling(window=100, min_periods=1).mean()

    # Calculate daily change and color
    new_df['diff'] = df[symbol].diff()
    new_df['color'] = new_df['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    if symbol == 'NIFTY 50':
        new_df.to_csv('test.csv')
    return new_df


def plot_stock_data(symbol, data):
    # Prepare plot data
    plot_data = data[data['Date'] > (pd.to_datetime('today') - pd.DateOffset(days=240))]

    # Setup subplot layout
    figure = make_subplots(rows=1, cols=1)

    # Plot candlestick chart
    figure.add_trace(go.Candlestick(x=plot_data['Date'],
                                    open=plot_data['Open'],
                                    high=plot_data['Close'],
                                    low=plot_data['Open'],
                                    close=plot_data['Close'],
                                    name='Price'))

    # Plot moving averages and volume
    for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
        figure.add_trace(go.Scatter(x=plot_data['Date'], y=plot_data[ma], marker_color=color, name=f'{ma}'))

    # Adjust Y-axis and layout for stock prices
    figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05])
    figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])  # hide weekends
    figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)

    return figure

def process_stocks():
    figure_html = open('screener_sectors.html', 'w')
    for symbol in df.columns.tolist()[1:]:
        data = calculate_technical_indicators_index(symbol)
        figure = plot_stock_data(symbol, data)
        figure_html.write(figure.to_html(full_html=False))
        # try:
        #     
        # except Exception as e:
        #     print(f"Error processing symbol {symbol}: {e.args}")

if __name__ == "__main__":
    process_stocks()
    print('Execution Completed!!!!')

Execution Completed!!!!
